<a href="https://colab.research.google.com/github/Good-lei/SpTransformer-Colab/blob/main/SpTransformer-test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 克隆仓库

In [1]:
!git clone https://github.com/Good-lei/SpTransformer-Colab.git

Cloning into 'SpTransformer-Colab'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 27 (delta 0), reused 0 (delta 0), pack-reused 24 (from 1)
Receiving objects: 100% (27/27), 43.18 MiB | 29.17 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
!pip install pyfaidx pyvcf3 pyensembl sinkhorn_transformer

# 1️⃣ 先降级 NumPy，防止与 pandas / pyarrow / torch 不兼容
!pip install numpy==1.26.4 --force-reinstall -q

# 3️⃣ 安装核心依赖（数据、模型、结构）
!pip install pandas pyarrow tqdm scikit-learn einops biopython -q

# 4️⃣ 安装 SpliceTransformer 特定依赖
!pip install axial-positional-embedding rpy2 -q

## 警告统统不用管

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 988.6/988.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.2/537.2 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 8.2 MB/s eta 0:00:00
  Created wheel for memoized-property: filename=memoized_property-1.0.3-py2.py3-none-any.whl size=4186 sha256=e5dbf7facc10c

## 下载权重

In [3]:
%mkdir /content/SpTransformer-Colab/model/weights
%cd /content/SpTransformer-Colab/model/weights
!pip install -q gdown
!gdown --id 1d8n4vHDSbXqpPc_JFEswLomSUDBgHvno -O SpTransformer_pytorch.ckpt
%cd /content/SpTransformer-Colab

/content/SpTransformer-Colab/model/weights
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1d8n4vHDSbXqpPc_JFEswLomSUDBgHvno
From (redirected): https://drive.google.com/uc?id=1d8n4vHDSbXqpPc_JFEswLomSUDBgHvno&confirm=t&uuid=9a5a959c-9d15-4e0a-8794-d02e0e159456
To: /content/SpTransformer-Colab/model/weights/SpTransformer_pytorch.ckpt
100% 120M/120M [00:02<00:00, 56.2MB/s]
/content/SpTransformer-Colab


## 下载基因组文件

In [4]:
# 1️⃣ 创建文件夹并进入
!mkdir -p /content/SpTransformer-Colab/data/data_package
%cd /content/SpTransformer-Colab/data/data_package

# 2️⃣ 下载 hg38.fa.gz
!wget -c https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

# 3️⃣ 解压并重命名为 hg38.fa
!gunzip -c hg38.fa.gz > hg38.fa

# 4️⃣ 下载 GENCODE v44 注释文件并重命名
!wget -c https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.annotation.gtf.gz -O hg38.annotation.gtf.gz

# 5️⃣ 查看结果
!ls -lh

%cd /content/SpTransformer-Colab

/content/SpTransformer-Colab/data/data_package
--2025-11-09 11:33:10--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938.09M  60.0MB/s    in 24s     

2025-11-09 11:33:35 (38.9 MB/s) - ‘hg38.fa.gz’ saved [983659424/983659424]

--2025-11-09 11:34:14--  https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.annotation.gtf.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49721965 (47M) [application/x-gzip]
Saving to: ‘hg38.annotation.gtf.gz’

hg38.annotation.gtf 

In [5]:
!pyensembl install --reference-name hg38 --annotation-name gencode.v38 --gtf "/content/SpTransformer-Colab/data/data_package/hg38.annotation.gtf.gz"

2025-11-09 11:34:54,643 - pyensembl.shell - INFO - Running 'install' for Genome(reference_name=hg38, annotation_name=gencode.v38, annotation_version=None, gtf_path_or_url=/content/SpTransformer-Colab/data/data_package/hg38.annotation.gtf.gz, transcript_fasta_paths_or_urls=, protein_fasta_paths_or_urls=)
2025-11-09 11:34:54,643 - pyensembl.database - INFO - Creating database: /content/SpTransformer-Colab/data/data_package/hg38.annotation.gtf.db
2025-11-09 11:34:54,643 - pyensembl.database - INFO - Reading GTF from /content/SpTransformer-Colab/data/data_package/hg38.annotation.gtf.gz
2025-11-09 11:36:35,285 - pyensembl.database - INFO - Skipping database index for {ccds_id}
2025-11-09 11:36:38,567 - datacache.database_helpers - INFO - Creating database /content/SpTransformer-Colab/data/data_package/hg38.annotation.gtf.db containing: CDS, gene, stop_codon, exon, transcript, start_codon
2025-11-09 11:36:38,568 - datacache.database - INFO - Running sqlite query: "CREATE TABLE CDS (source TE

In [7]:
%cd /content/SpTransformer-Colab
!python /content/SpTransformer-Colab/sptransformer.py --input data/example/input38.vcf --output output_unc13a.tsv --reference hg38 --raw_score True

/content/SpTransformer-Colab
Please ignore this warning if you are using hg38
hg19 fasta not found
torch device:cpu
Reading vcf file:data/example/input38.vcf
1it [00:19, 19.18s/it]
INFO:rpy2.rinterface_lib.embedded:Embedded R ended.
INFO:rpy2.rinterface_lib.embedded:Embedded R already ended.
